1. 

In [1]:
#!pip install tensorflow tensorflow-gpu opencv-python mediapipe sklearn matplotlib

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

2.

In [3]:
mp_holistic = mp.solutions.holistic # holistic models
mp_drawing = mp.solutions.drawing_utils # drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # color conversion
    image.flags.writeable = False # image no longer writeable
    results = model.process(image) # make prediction
    image.flags.writeable = True # image writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # color conversion
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)


In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [7]:
cap = cv2.VideoCapture(0)
# set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # read feed
        ret, frame = cap.read()

        # make detection
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # draw landmarks
        draw_styled_landmarks(image, results)


        # show
        cv2.imshow('OpenCV Feed', image)

        # break
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

3.

In [8]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [9]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [10]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [11]:
result_test = extract_keypoints(results)

In [12]:
result_test

array([ 0.39391124,  0.36666888, -1.33799529, ...,  0.        ,
        0.        ,  0.        ])

In [13]:
468*3+33*4+21*3+21*3

1662

In [14]:
np.save('0', result_test)

In [15]:
np.load('0.npy')

array([ 0.39391124,  0.36666888, -1.33799529, ...,  0.        ,
        0.        ,  0.        ])

4.

In [16]:
import csv

In [17]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

501

In [18]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [19]:
landmarks

['class',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 '

In [21]:
with open('dataset.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [22]:
class_name = "Z"

In [23]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)

        # Draw landmarks
        draw_styled_landmarks(image, results)
       
        # Export koordinat
        try:
            # Ambil Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Ambil Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Satukan baris
            row = pose_row+face_row
            
            # Tambah class name 
            row.insert(0, class_name)
           
    
            # Export ke CSV
            with open('dataset.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

5

5.1

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [25]:
df = pd.read_csv('dataset.csv')

In [26]:
dept_emp_num = df.groupby('class')['class'].count()
print (dept_emp_num)
df.shape

class
Z    34
Name: class, dtype: int64


(34, 2005)

In [27]:
startline, endline = 52000, 52053 
filename1= 'dataset.csv'
with open(filename1, 'r', newline='') as f:
    content = [row for i,row in enumerate(csv.reader(f), 1)
                    if i not in range(startline, endline+1)]

filename2 = 'dataset.csv'  
with open(filename2, 'w', newline='') as f:
    csv.writer(f).writerows(content)

print('lines deleted')

lines deleted


In [28]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Z,0.324761,0.387148,-1.065416,0.999401,0.360216,0.289722,-1.019895,0.998981,0.383031,...,-0.010619,0.0,0.403682,0.286473,0.020364,0.0,0.410657,0.275342,0.021223,0.0
1,Z,0.325185,0.390305,-1.169538,0.999460,0.360134,0.291320,-1.109233,0.999081,0.382176,...,-0.016338,0.0,0.388186,0.295159,0.012844,0.0,0.395228,0.283368,0.013414,0.0
2,Z,0.325188,0.399770,-1.150297,0.999513,0.357435,0.300253,-1.098096,0.999171,0.376976,...,-0.007861,0.0,0.375810,0.320191,0.023658,0.0,0.382976,0.308073,0.024767,0.0
3,Z,0.323433,0.401820,-0.878797,0.999561,0.351558,0.304610,-0.832450,0.999252,0.369613,...,-0.005184,0.0,0.368169,0.324062,0.030921,0.0,0.375724,0.313578,0.032430,0.0
4,Z,0.316307,0.409418,-0.877578,0.999605,0.342804,0.312536,-0.828799,0.999326,0.361295,...,-0.010623,0.0,0.367140,0.324020,0.021597,0.0,0.374460,0.313363,0.022851,0.0


In [29]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
29,Z,0.230482,0.476520,-0.995494,0.999951,0.261034,0.369190,-0.961042,0.999916,0.284123,...,-0.017525,0.0,0.323565,0.347691,0.007099,0.0,0.330372,0.335531,0.007795,0.0
30,Z,0.272746,0.451504,-1.056326,0.999955,0.301666,0.348519,-1.007766,0.999921,0.322866,...,-0.014780,0.0,0.370320,0.337993,0.013515,0.0,0.377611,0.323955,0.014786,0.0
31,Z,0.332409,0.437249,-0.975621,0.999940,0.364154,0.340733,-0.928891,0.999896,0.386646,...,-0.001038,0.0,0.426751,0.336875,0.038051,0.0,0.434448,0.323612,0.040081,0.0
32,Z,0.364932,0.435167,-1.107600,0.999919,0.401717,0.338950,-1.067343,0.999859,0.425983,...,0.002274,0.0,0.466322,0.315636,0.041681,0.0,0.475280,0.301931,0.043900,0.0
33,Z,0.370386,0.432521,-1.099459,0.999892,0.411090,0.335652,-1.060098,0.999815,0.436805,...,0.000348,0.0,0.480981,0.318680,0.036267,0.0,0.489485,0.307046,0.037995,0.0


In [30]:
df[df['class']=='Z']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Z,0.324761,0.387148,-1.065416,0.999401,0.360216,0.289722,-1.019895,0.998981,0.383031,...,-0.010619,0.0,0.403682,0.286473,0.020364,0.0,0.410657,0.275342,0.021223,0.0
1,Z,0.325185,0.390305,-1.169538,0.999460,0.360134,0.291320,-1.109233,0.999081,0.382176,...,-0.016338,0.0,0.388186,0.295159,0.012844,0.0,0.395228,0.283368,0.013414,0.0
2,Z,0.325188,0.399770,-1.150297,0.999513,0.357435,0.300253,-1.098096,0.999171,0.376976,...,-0.007861,0.0,0.375810,0.320191,0.023658,0.0,0.382976,0.308073,0.024767,0.0
3,Z,0.323433,0.401820,-0.878797,0.999561,0.351558,0.304610,-0.832450,0.999252,0.369613,...,-0.005184,0.0,0.368169,0.324062,0.030921,0.0,0.375724,0.313578,0.032430,0.0
4,Z,0.316307,0.409418,-0.877578,0.999605,0.342804,0.312536,-0.828799,0.999326,0.361295,...,-0.010623,0.0,0.367140,0.324020,0.021597,0.0,0.374460,0.313363,0.022851,0.0
5,Z,0.312578,0.410178,-0.889382,0.999642,0.337147,0.312804,-0.842476,0.999389,0.355675,...,-0.008361,0.0,0.364191,0.326231,0.030682,0.0,0.371648,0.315341,0.032443,0.0
6,Z,0.306057,0.412481,-1.075667,0.999677,0.330460,0.315436,-1.014288,0.999447,0.349522,...,-0.011930,0.0,0.362938,0.329122,0.026096,0.0,0.370325,0.317550,0.027866,0.0
7,Z,0.306619,0.415712,-0.968012,0.999707,0.330869,0.318363,-0.915280,0.999500,0.349134,...,-0.011521,0.0,0.359490,0.328655,0.022394,0.0,0.366418,0.318184,0.023739,0.0
8,Z,0.298985,0.423575,-1.134086,0.999736,0.324090,0.323564,-1.076863,0.999547,0.343281,...,-0.008640,0.0,0.350075,0.330861,0.023004,0.0,0.357398,0.318977,0.024117,0.0
9,Z,0.292799,0.429327,-1.209954,0.999762,0.317719,0.327618,-1.141670,0.999591,0.338515,...,-0.016145,0.0,0.345414,0.328435,0.009351,0.0,0.352173,0.318934,0.009820,0.0


In [31]:
X = df.drop('class', axis=1) # features akan menjadi semua nilai koord
y = df['class'] # target = label kelas

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [33]:
X_test

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
4,0.316307,0.409418,-0.877578,0.999605,0.342804,0.312536,-0.828799,0.999326,0.361295,0.307575,...,-0.010623,0.0,0.367140,0.324020,0.021597,0.0,0.374460,0.313363,0.022851,0.0
13,0.189996,0.452452,-1.175017,0.999836,0.219123,0.350437,-1.149068,0.999719,0.247871,0.338625,...,-0.011947,0.0,0.274110,0.331569,0.018929,0.0,0.279973,0.320446,0.019673,0.0
14,0.195333,0.454502,-1.157048,0.999851,0.223178,0.352061,-1.125771,0.999745,0.250880,0.340092,...,-0.014088,0.0,0.273631,0.336098,0.015225,0.0,0.279660,0.323435,0.016287,0.0
27,0.208369,0.473289,-1.056035,0.999944,0.235211,0.365573,-1.033769,0.999902,0.259835,0.355190,...,-0.016081,0.0,0.290731,0.354473,0.010136,0.0,0.297094,0.342115,0.010700,0.0
8,0.298985,0.423575,-1.134086,0.999736,0.324090,0.323564,-1.076863,0.999547,0.343281,0.317571,...,-0.008640,0.0,0.350075,0.330861,0.023004,0.0,0.357398,0.318977,0.024117,0.0
1,0.325185,0.390305,-1.169538,0.999460,0.360134,0.291320,-1.109233,0.999081,0.382176,0.287854,...,-0.016338,0.0,0.388186,0.295159,0.012844,0.0,0.395228,0.283368,0.013414,0.0
0,0.324761,0.387148,-1.065416,0.999401,0.360216,0.289722,-1.019895,0.998981,0.383031,0.285909,...,-0.010619,0.0,0.403682,0.286473,0.020364,0.0,0.410657,0.275342,0.021223,0.0
18,0.204398,0.472222,-1.161596,0.999896,0.232675,0.369313,-1.136352,0.999821,0.259027,0.358888,...,-0.016727,0.0,0.289351,0.346069,0.009689,0.0,0.296547,0.334138,0.010205,0.0
7,0.306619,0.415712,-0.968012,0.999707,0.330869,0.318363,-0.915280,0.999500,0.349134,0.313081,...,-0.011521,0.0,0.359490,0.328655,0.022394,0.0,0.366418,0.318184,0.023739,0.0
3,0.323433,0.401820,-0.878797,0.999561,0.351558,0.304610,-0.832450,0.999252,0.369613,0.300089,...,-0.005184,0.0,0.368169,0.324062,0.030921,0.0,0.375724,0.313578,0.032430,0.0


5.2

In [34]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.ensemble import RandomForestClassifier

In [35]:
pipelines = {

    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
}

In [36]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [37]:
fit_models

{'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())])}

In [38]:
fit_models['rf'].predict(X_test)

array(['Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z'],
      dtype=object)

5.3

In [39]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix # Accuracy metrics 
import pickle 

In [40]:
for algo, model in fit_models.items():
    y_prediksi = model.predict(X_test) #sebagai y prediksi
    print(algo, classification_report(y_test, y_prediksi))

rf               precision    recall  f1-score   support

           Z       1.00      1.00      1.00        11

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11



In [41]:
fit_models['rf'].predict(X_test)

array(['Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z'],
      dtype=object)

In [42]:
y_train

6     Z
2     Z
31    Z
26    Z
30    Z
33    Z
28    Z
29    Z
21    Z
5     Z
16    Z
22    Z
32    Z
11    Z
9     Z
23    Z
17    Z
15    Z
25    Z
24    Z
20    Z
12    Z
19    Z
Name: class, dtype: object

In [43]:
with open('model.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

6.

In [44]:
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

In [45]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [46]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Menyatukan baris
        row = pose_row+face_row
            
        # Export koordinat
        try:
            # Mengambil Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Mengambil Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Menyatukan baris
            row = pose_row+face_row
            
            
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            
            # Ambil koordinat telinga
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display prob
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Z [1.]


c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Z [1.]


c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...

Z [1.]


c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Z [1.]


c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Z [1.]


In [47]:
tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))

(314, 156)